In [1]:
import dspy
import os
from dspy.teleprompt import *
import random
import math
import numpy as np
from python_tsp.exact import solve_tsp_dynamic_programming
import re

os.environ['TOGETHER_API_KEY'] = '35ba5bebf6288e43fdc8989965161592e3335d7067c772c0c6995cdc0e60cd88'
os.environ['TOGETHER_API_BASE'] = 'https://api.together.xyz/v1'

In [2]:
# constants
NUM_CITIES = 10
TRAIN_INSTANCES = 150
TEST_INSTANCES = 100
CITIES = " ".join(map(str, list(np.arange(NUM_CITIES))))
NUM_THREADS = 5

In [3]:
# helper functions
def euclidean_distance(point1, point2):
    return math.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

def calc_path_distance(path, distances):
    total_distance = 0
    if len(path) < 2:
        return 0
    for i in range(len(path)-1):
        total_distance += distances[path[i]][path[i + 1]]
    total_distance += distances[0][path[len(path)-1]]
    return total_distance

def make_graphs(num_instances, num_cities):
    x_range = (-20, 20)
    y_range = (-20, 20)

    distanceList = []
    for _ in range(num_instances):
        coordinates = [(random.uniform(*x_range), random.uniform(*y_range)) for _ in range(num_cities)]
        distance_matrix = [[euclidean_distance(coordinates[i], coordinates[j]) for j in range(num_cities)] for i in range(num_cities)]
        distanceList.append(np.array(distance_matrix))    
    return distanceList

def make_dataset(distanceList):
    dataset = []
    for matrix in distanceList:
        permutation, distance = solve_tsp_dynamic_programming(matrix)
        data_point = {
            "distance_matrix": matrix.tolist(),
            "optimal_route": permutation,
            "optimal_distance": distance
        }
        dataset.append(data_point)
    return dataset

def makeDSPYExamples(dataset):
    exampleList = []
    for example in dataset:
        distances = "\n".join([" ".join(map(str, row)) for row in example["distance_matrix"]])
        route = " ".join(map(str, example["optimal_route"]))
        exampleObj = dspy.Example(cities=CITIES, distances=distances, route=route).with_inputs("cities", "distances")
        exampleList.append(exampleObj)
    return exampleList

def random_baseline(distances):
    numbers = list(range(NUM_CITIES))
    random.shuffle(numbers)
    curr = numbers[0]
    for i in range(len(numbers)):
        if numbers[i] == 0:
            numbers[i] = curr
            break
    numbers[0] = 0
    path_length = calc_path_distance(path=numbers, distances=distances)
    return numbers, path_length

In [4]:
# Train set:
train_dl = make_graphs(TRAIN_INSTANCES, NUM_CITIES)
train_ds = make_dataset(train_dl)
tsp_trainset = makeDSPYExamples(train_ds)
print(tsp_trainset[0])

Example({'cities': '0 1 2 3 4 5 6 7 8 9', 'distances': '0.0 24.576983609195434 38.81009568709092 18.245165173189537 31.481052056617273 29.442638039483782 11.882745006002425 9.659164797348945 21.095957354333077 35.380813151398556\n24.576983609195434 0.0 20.964341161504212 6.7376227659094985 7.290359296993905 5.059488396089626 12.710147081759468 14.930301896854264 4.5317124994465745 34.441020858463\n38.81009568709092 20.964341161504212 0.0 26.083781240785925 16.296490217677075 17.815165153270833 29.270271486335208 30.947377314404125 25.08598522202352 24.6694528404397\n18.245165173189537 6.7376227659094985 26.083781240785925 0.0 14.018597526711897 11.797056228192034 6.449151662209844 8.634969847187811 2.8743145791046385 34.9269936485837\n31.481052056617273 7.290359296993905 16.296490217677075 14.018597526711897 0.0 2.267720467218318 19.738314234709296 21.92832458194677 11.726437725040284 34.859991298932755\n29.442638039483782 5.059488396089626 17.815165153270833 11.797056228192034 2.26772

In [5]:
# Test set:
test_dl = make_graphs(TEST_INSTANCES, NUM_CITIES)
test_ds = make_dataset(test_dl)
tsp_testset = makeDSPYExamples(test_ds)

In [6]:
llama = dspy.Together(model="meta-llama/Meta-Llama-3-70B")
dspy.configure(lm=llama)

In [7]:
class TSP(dspy.Module):
    def __init__(self):
        super().__init__()
        self.make_route = dspy.Predict(TSPSignature)
        
    def forward(self, cities, distances):
        pred_route = self.make_route(cities=cities, distances=distances)
        # print(pred_route)
        return pred_route
    
class TSPSignature(dspy.Signature):
    """Generate a route, starting at city 0, visiting all cities once, minimizing distance traveled. The distance from the final node to city 0 will be added."""
    cities = dspy.InputField()
    distances = dspy.InputField()
    route = dspy.OutputField()

def extract_route(route, N=NUM_CITIES):
    # Extract the first N numbers from the route string
    numbers = re.findall(r'\d+', route)[:N]
    
    # Convert the numbers to integers
    numbers = list(map(int, numbers))
    
    return numbers

def eval_tour(cities, route, distances):
    distances_matrix = np.array([list(map(float, row.split())) for row in distances.split('\n')])
    # print(distances)
    # print(distances_matrix)
    try:
        route = extract_route(route) # make it a list of ints
        # print(route)
    except ValueError:
        raise ValueError(f"Invalid route: {route}")
    if len(route) != len(distances_matrix):
        raise ValueError(f"Route length {len(route)} does not match number of cities {len(distances_matrix)}")

    # make it so that it's the difference between the two
    total_distance = sum(distances_matrix[route[i]][route[i+1]] for i in range(len(route) - 1))
    total_distance += distances_matrix[route[-1]][route[0]]
    return total_distance

# validation function for the TSP
def metric(example, pred, trace=None):
    try:
        distance = eval_tour(example.cities, pred.route, example.distances)
        return -distance  # Return negative distance to maximize the metric
    except ValueError as e:
        dspy.logger.error(e)
        return float('-inf')
    

In [8]:
# DSPy optimizer to improve the TSP solution
# config = dict(
#     max_bootstrapped_demos=4,   # Number of bootstrapped demonstrations
#     max_labeled_demos=4,        # Number of labeled demonstrations
#     num_candidate_programs=10,  # Number of candidate programs to evaluate
#     num_threads=4               # Number of threads for parallel evaluation
# )
# teleprompter = BootstrapFewShotWithRandomSearch(metric=metric, **config)

teleprompter = LabeledFewShot(k=7)
compiled_tsp = teleprompter.compile(TSP(), trainset=tsp_trainset)

In [9]:
evaluater = Evaluate(devset=tsp_testset, metric=metric, num_threads=NUM_THREADS, display_progress=True, display_table=0)
evaluater(compiled_tsp)

Average Metric: -6530.7648439304 / 33  (-19790.2):  33%|▎| 33/100 [01:58<03:30, 2024-07-19T19:31:09.315997Z [warning  ] Received SIGINT. Cancelling evaluation. [dspy.evaluate.evaluate] filename=evaluate.py lineno=91
2024-07-19T19:31:09.534905Z [warning  ] Received SIGINT. Cancelling evaluation. [dspy.evaluate.evaluate] filename=evaluate.py lineno=91


KeyboardInterrupt: 

In [ ]:
test_example = tsp_testset[0]
numerical_test_example = test_ds[0]
print(numerical_test_example)

In [ ]:
predicted_result = compiled_tsp(cities=test_example.cities, distances=test_example.distances)

predicted_route = predicted_result.route

predicted_distance = eval_tour(test_example.cities, predicted_route, test_example.distances)

print(f"Predicted route: {predicted_route}")
print(f"Total distance of the predicted route: {predicted_distance}")

optimal_route = test_example.route
optimal_distance = eval_tour(test_example.cities, optimal_route, test_example.distances)
print(f"Optimal route: {optimal_route}")
print(f"Total distance of the optimal route: {optimal_distance}")

In [ ]:
print(numerical_test_example)
path, distance = random_baseline(numerical_test_example["distance_matrix"])
print(f"path is {path}")
print(f"distance is {distance}")

Random baseline eval:

In [ ]:
total_dis = 0
for i in range(TEST_INSTANCES):
    curr_example = test_ds[i]
    _, distance = random_baseline(curr_example["distance_matrix"])
    total_dis += distance
print(f"(RANDOM) total distance is {total_dis}")
print(f"(RANDOM) average distance is {total_dis/TEST_INSTANCES}")

Model eval:

In [ ]:
print("(MODEL) average distance is 197")

Optimal route eval:

In [ ]:
total_dis = 0
for i in range(TEST_INSTANCES):
    curr_example = test_ds[i]
    total_dis += curr_example["optimal_distance"]
print(f"(OPTIMAL) total distance is {total_dis}")
print(f"(OPTIMAL) average distance is {total_dis/TEST_INSTANCES}")